In [1]:
import numpy as np
import pandas as pd
import os

from tqdm import tqdm

from utils.ffmpeg_processor import ffmpegProcessor
from utils.audio import melspectrogram, show_melspectrogram, save_melspectrograms
from utils.encoder import FakeRealEncoder

Using TensorFlow backend.


In [2]:
fase = 'GEN'

In [3]:
if fase == 'GEN':
    df = pd.DataFrame()
    for root, dirs, files in os.walk('../videos', topdown=False):
        for name in dirs:
            print(name)
            dfdir = pd.read_json('../videos/' + name + '/metadata.json')
            dfdir = dfdir.T
            dfdir['dir'] = name
            df = df.append(dfdir)

    df["processed"] = 'False'
    df.to_csv('../data/audio.csv',index_label='video')

dfdc_train_part_21
dfdc_train_part_20
dfdc_train_part_14
dfdc_train_part_48
dfdc_train_part_3
dfdc_train_part_2


In [4]:
df = pd.read_csv('../data/audio.csv')

In [5]:
threshold = 0.08
n_chunks = 10
len_chunks = 44100 * 2 
inc_chunk = (441236 - len_chunks) // (n_chunks - 1)

In [6]:
ap = ffmpegProcessor()
le = FakeRealEncoder()

real_videos_processed = []

df = df.query('label == "FAKE"')
for index, row in tqdm(df.iterrows(), total=len(df)):
    af = ap.extract_audio(f"../videos/{row.dir}/{row.video}")
    ar = ap.extract_audio(f"../videos/{row.dir}/{row.original}")
    l = min(len(ar),len(af))
    
    if np.max(af[:l]-ar[:l]) > threshold:
        af = np.resize(af, (n_chunks,len_chunks))
        save_melspectrograms(af, n_chunks, f'fake_{row.video}', le.to_categorical(['FAKE']))
        # add real video if not processed
        if row.original not in real_videos_processed:
            ar = np.resize(ar, (n_chunks,len_chunks))
            save_melspectrograms(ar, n_chunks, f'real_{row.original}', le.to_categorical(['REAL']))
            real_videos_processed.append(row.original)


100%|██████████| 10582/10582 [1:08:51<00:00,  2.98it/s]


## Test

In [ ]:
from utils.utils import npz_in_folder
files = npz_in_folder('../data/audio')

In [ ]:
data = np.load(files[42])
log_S=data['data']
label=data['label']
data.close()

In [ ]:
log_S.shape, label.shape

In [ ]:
show_melspectrogram(log_S)